In [62]:
 # This file will use the random forest algorithm to classify the text in the dataset to predict the polarity of the text.
    # The dataset is a collection of reddit post that is manually labeled as positive or negative.

# Import the necessary libraries
# Data Processing
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Modelling 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns


[nltk_data] Downloading package stopwords to /Users/chris/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Dataset Preparation

In [63]:
# Import the dataset
R_df = pd.read_csv('../Data/Relavance.csv')
P_df = pd.read_csv('../Data/Polarity.csv')

# # Remove special characters from Title and Data columns such as as ð,Ÿ,˜,â,€,™, and ðŸ˜
R_df['Data'] = R_df['Data'].str.replace('ðŸ˜','')
R_df['Data'] = R_df['Data'].str.replace('ðŸ‘€','')
# Remove ðŸ
P_df['Data'] = P_df['Data'].str.replace('ðŸ˜','')
P_df['Data'] = P_df['Data'].str.replace('ðŸ‘€','')

# # Replace all the NaN values with empty string
R_df['Data'] = R_df['Data'].fillna('')
R_df['Relavance'] = R_df['Relavance'].fillna('')
P_df['Data'] = P_df['Data'].fillna('')    
P_df['Polarity'] = P_df['Polarity'].fillna('')

# Replace \n with empty string
R_df['Data'] = R_df['Data'].str.replace('\n','')
P_df['Data'] = P_df['Data'].str.replace('\n','')

# Flags to run either the relavance or polarity model
relavance = False
polarity = True

# print(df.head())
# print(df.shape)
# print(R_df.info())
# print(R_df.columns.values)

In [64]:
# Preprocessing Function for each data
def preprocess_bows(df, df_size):
    data_soup = BeautifulSoup(df)
    data_text = data_soup.get_text()
    data_letters_only = re.sub("[^a-zA-Z]", " ", data_text).lower()
    data_words = data_letters_only.split()
    stops = set(stopwords.words("english"))
    meaningful_words = [w for w in data_words if not w in stops]

    if((i)%100 == 0):
        print("Cleaned %d %d data (%d %%)." % (i, df_size, (i/df_size)*100))
    return(" ".join(meaningful_words))

In [65]:
# Preprocess the data
R_df_size = R_df['Data'].size
P_df_size = P_df['Data'].size

if relavance:
    for i in range(R_df_size):
        R_df['Data'][i] = preprocess_bows(R_df['Data'][i], R_df_size)

if polarity:
    for i in range(P_df_size):
        P_df['Data'][i] = preprocess_bows(P_df['Data'][i], P_df_size)

Cleaned 0 4300 data (0 %).


/var/folders/2q/bjqpws1153b0_y18lhffmh_m0000gn/T/ipykernel_71366/771688903.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  P_df['Data'][i] = preprocess_bows(P_df['Data'][i], P_df_size)


Cleaned 2000 4300 data (46 %).
Cleaned 4000 4300 data (93 %).


In [72]:
# Prepare Training Features

cv = CountVectorizer( analyzer='word', tokenizer=None, preprocessor=None, stop_words=None, max_features=5000)
# X = P_df.drop('Polarity', axis=1)
X = P_df['Data']
y = P_df['Polarity']

# Split the dataset into training and testing set
X_P_train, X_P_test, y_P_train, y_P_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(X_P_train.shape, X_P_test.shape, y_P_train.shape, y_P_test.shape)

(3010,) (1290,) (3010,) (1290,)


In [73]:
# Convert train, validation and test data to vectors
print(X_P_train.shape)
X_P_train = cv.fit_transform(X_P_train)
print(X_P_train.shape)
X_P_train = X_P_train.toarray()

(3010,)
(3010, 5000)


In [74]:
# X_P_test = cv.fit_transform(X_P_test)
# print(X_P_test.shape)
# # X_P_test = X_P_test.toarray()
# print(X_P_test.shape)

In [76]:
# X_test_text = cv.transform()
vocab = cv.get_feature_names_out()
print(vocab)

['aaml' 'ab' 'abilities' ... 'zipper' 'zverev' 'zverevs']


distribution = np.sum(X_P_train, axis=0)

print("Printing first 100 vocab-dist pairs:")

for tag, count in zip(vocab[:100], distribution[:100]):
    print(count, tag)

In [80]:
# Encode the labels
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_P_train = le.fit_transform(y_P_train)
y_P_test = le.fit_transform(y_P_test)

# Fitting and Training the model
rf = RandomForestClassifier()
rf.fit(X_P_train, y_P_train)

RandomForestClassifier()

In [86]:
# Predicting the test set results
y_P_pred = rf.predict(X_P_test)
print("Accuracy: ", accuracy_score(y_P_test, y_P_pred))

ValueError: could not convert string to float: 'rolex submariner hulk lv super condition full set box amp papers rolex submariner hulk lv'